In [ ]:
##
#1 - go to car postings pages
#2 - for each page, go to the url for each post
#3 - on each car page, grab all the car attributes on the bottom left quadrant
#4 - append to dataframe
#5 - perform cleaning steps
#6 - visualize
##

# <font color='red'> Import libraries </font>

In [ ]:
# work with dataframes
import pandas as pd
import numpy as np

# we'll use this to store the a dataframe to csv at a later stage
import csv

# make HTTP requests to a specified URL
import requests

# web scraping library
from bs4 import BeautifulSoup

# time management
import time
from random import randint
# from collections import Counter

# regular expressions
import re

In [ ]:
# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
%matplotlib inline

# Set the default color cycle for charts
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["r", "#e94cdc", "0.7"]) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#### Use BeautifulSoup to scrape Craigslist DFW Used Cars

In [ ]:
# a link to each vehicle post will be stored here
links = []

#  all pages in a city will be stored in this list
list_of_cities = []

# list of city names we want to get data for
cities = ['dallas','chicago', 'newyork', 'sfbay', 'losangeles', \
        'houston', 'phoenix', 'philadelphia', 'sanantonio', 'washingtondc',\
       'boston', 'nashville', 'atlanta', 'miami', 'seattle']


for city in cities:
    # each city has approxiamtely 1800 pages for the "cars for sale by owner" category
    # we'll keep track of these pages with page_number variable below
    page_number = 1

    # this while loop cycles through all 1800 pages
    while page_number <= 1800:
        # city_link variable takes a a different city name from the cities every time through the loop
        city_link = "https://" + str(city) + ".craigslist.org/d/cars-trucks-by-owner/search/cto?s=" + \
                                str(page_number) + "&hasPic=1"

        # we                         
        list_of_cities.append(city_link)
        page_number +=120

# URLs counter
car_urls = 1      
for each_city_page in list_of_cities:
    links_in_each_city_page = requests.get(each_city_page)
    # parse html object from page to BS object
    soup = BeautifulSoup(links_in_each_city_page.content, 'html.parser')

    try:
        #get the macro-container for the car posts for that page
        posts = soup.find_all('a', class_= 'result-image gallery')

        # get all the html links in the page and append them to a list
        for link in posts:
            l = link.get('href')
            links.append(l)
                
    except:
        pass

    # this code just helps us keep count of the looping progress                     
    if car_urls % 5 == 0:
        city_link = l.strip()
        start = city_link.find("//") + len("//")
        end = city_link.find(".")
        city_string = city_link[start:end]
        print('Number of pages returned --> ' + str(car_urls) + '---' + city_string)

    # we add a sleep timer to manage our server requests
    time.sleep(randint(0,1))
    car_urls +=1

Number of pages returned --> 5---dallas
Number of pages returned --> 10---dallas
Number of pages returned --> 15---dallas
Number of pages returned --> 20---chicago
Number of pages returned --> 25---chicago
Number of pages returned --> 30---chicago
Number of pages returned --> 35---newyork
Number of pages returned --> 40---newyork
Number of pages returned --> 45---newyork
Number of pages returned --> 50---sfbay
Number of pages returned --> 55---sfbay
Number of pages returned --> 60---sfbay
Number of pages returned --> 65---losangeles
Number of pages returned --> 70---losangeles
Number of pages returned --> 80---houston
Number of pages returned --> 85---houston
Number of pages returned --> 90---houston
Number of pages returned --> 95---phoenix
Number of pages returned --> 100---phoenix
Number of pages returned --> 105---phoenix
Number of pages returned --> 110---philadelphia
Number of pages returned --> 115---philadelphia
Number of pages returned --> 120---philadelphia
Number of pages re

In [ ]:
# save links to csv
df = pd.DataFrame(links)
df.to_csv("./links.csv", sep=',',index=False)

In [ ]:
# read links from csv
links = pd.read_csv('./links.csv',names=['https'])
links = links['https'][1:]
print("links returned --> " , len(links))
links

links returned -->  225


1      https://dallas.craigslist.org/d/cars-trucks-by...
2      https://dallas.craigslist.org/d/cars-trucks-by...
3      https://dallas.craigslist.org/d/cars-trucks-by...
4      https://dallas.craigslist.org/d/cars-trucks-by...
5      https://dallas.craigslist.org/d/cars-trucks-by...
                             ...                        
221    https://Seattle.craigslist.org/d/cars-trucks-b...
222    https://Seattle.craigslist.org/d/cars-trucks-b...
223    https://Seattle.craigslist.org/d/cars-trucks-b...
224    https://Seattle.craigslist.org/d/cars-trucks-b...
225    https://Seattle.craigslist.org/d/cars-trucks-b...
Name: https, Length: 225, dtype: object

In [ ]:
# justa counter
count = 0

# store vehicle details in this list
cars = []

#loop over all links in the list    
for link in links:
    # make HTTP requests
    each_page = requests.get(link)
    # The sleep function can help you to avoid the server to be overloaded with too many requests in a very short period of time.
    time.sleep(randint(1,2))
    # store the BS object in a variable
    page_soup = BeautifulSoup(each_page.content, 'html.parser')

    # loop over each link and store car details
    car_details = []
    try:
        # find price attribute and store in car details
        car_details.append(page_soup.find('span', class_="price").text)

        # find date time and append to car details
        for span in page_soup.find_all('span', recursive=True):
            if not span.attrs.values():
                car_details.append(span.text)
        car_details.append("date time: " + page_soup.find('time', class_="date timeago")\
                                         .text.strip().replace(':',';'))

        # find date city name and append to car details
        city = link.strip()
        start = city.find("//") + len("//")
        end = city.find(".")
        substring = city[start:end]
        car_details.append('city:' + substring)

        # find geo coordinates and append to car details
        geos = page_soup.findAll("div", {"class": "mapbox"})
        lat = geos[0].contents[1].get('data-latitude')
        car_details.append('lat:' + lat.strip())
        long = geos[0].contents[1].get('data-longitude')
        car_details.append('long:' + long.strip())

        # find post body and append to car details
        post_body = page_soup.find(attrs={'id' : 'postingbody'}).contents[2]
        # remove non ascii characters from post bosy
        car_details.append('post_body:' + re.sub("[^0-9a-zA-Z]+", " ", post_body))

        # find postID and append to car details / We'll use this to assign labels to images later
        car_details.append('pID:' + link.strip().replace('html','').replace('.','').split('/')[-1])
    except:
        pass    
    
    # perform some basic cleanup and store in clean
    clean = []
    for string in car_details:
        # this attribute came without a label. Assign one.
        if string == car_details[1]:
            clean.append('year make model: ' + string)
        # clean up price text from $9,999 --> 9999
        if string == car_details[0]:
            clean.append('price: ' + string.replace(',','').replace('$',''))
        else:
            clean.append(string)

            
    # some attributes came without labels. Drop those.
    car_final = []
    for s in clean:
        if ':' in s:
            car_final.append(s)
            
    # append clean attributes for each vehicle to car list
    cars.append(car_final)
    count += 1

    # just a counter to keep track of the loop
    if count % 100 == 0:
        print('loop # -> ',count)

In [ ]:
# method to strip() the keys and values after splitting in order to trim white-space.
def list_to_dict(rlist):
    return dict(map(lambda s : map(str.strip, s.split(':')), rlist))



# create a dictionary for label:value for each car attribute
car_dicts = []
for car in cars:
    car_dict = list_to_dict(car)
    car_dicts.append(car_dict)

In [ ]:
# create an Empty DataFrame object
dfs = pd.DataFrame()

for item in car_dicts:
    df = pd.DataFrame.from_dict(item,orient='index').transpose()
    #concatenate each new df from the loop into the parent df
    dfs= pd.concat([dfs,df], axis=0, ignore_index=True, sort=True)
    #clean duplicate year in 'year make model'
    dfs['year_c make model'] = dfs['year make model'].str.replace(r'\b(\w+)(\s+\1)+\b', r'\1', regex=True)

In [ ]:
dfs.to_csv('car_data.csv', sep='\t', encoding='utf-8')
dfs.to_csv('B:/DS/car_data.csv', sep='\t', encoding='utf-8')